**Imports**

In [ ]:
# Imports und Funktions-Imports
# Standard- und externe Bibliotheken
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt

# Eigene Funktions-Module für Daten, Berechnung und Steuerung
from functions.retro_data.weather_data import get_weather
from functions.retro_data.spotprice_data import get_spotprice
from functions.retro_data.consumption_data import get_demand

from functions.forecast.spotprice_forecast_data import get_spotprice_forecast
from functions.forecast.weather_forecast_data import get_weather_forecast
from functions.forecast.weather_real_data import get_weather_real
from functions.forecast.consumption_forecast_data import get_demand_forecast

from functions.calculations.calculations import calculate_energy
from functions.calculations.kpis_valid import calculate_kpis_valid
from functions.deciding_functions.optimized import optimize_energy_flow
from functions.deciding_functions.reinforcementlearning.reinforcementlearning_train import reinforcement_learning_train
from functions.deciding_functions.reinforcementlearning.reinforcementlearning_test import reinforcement_learning_test
from functions.deciding_functions.rulebased import rule_based_energy_flow
from functions.results.results import get_results

**Definition des Green Energy Ecosystems**

In [ ]:
# Systemparameter und Annahmen
# Hier werden alle relevanten Systemparameter für PV, Wind, Batterie, Verbraucher etc. gesetzt.
# Werte können für Szenarien einfach angepasst werden.

# Photovoltaik
solar_peak_power = 38  # Spitzenleistung Gesamtsystem PV [MW]
num_modules = 114980 #Anzahl Solarmodule
area_per_module = 1.57458  # Fläche pro Modul [m²] (Annahme Heckert Solar NeMo® 2.0 330 Wp)
module_efficiency = 0.196  # Wirkungsgrad der Solarmodule (Annahme Heckert Solar NeMo® 2.0 330 Wp)
tilt_angle = 45  # Neigungswinkel der Solarpanels [°] (Annahme)
latitude=51.751111 # Breitengrad
longitude=4.208611 # Längengrad
azimuth=180 # Azimut [°]
albedo=0.2 # Albedo

# Windkraft 
wind_peak_power = 22   # Spitzenleistung Gesamtsystem Windkraft [MW]
r = 60 # Rotor Radius [m]
h = 90 # Höhe Windrad [m]
turbine_efficiency = 0.5  # Wirkungsgrad der Windkraftanlagen (Annahme)
cut_in = 2.5  # Einschaltgeschwindigkeit [m/s]
cut_out = 28  # Abschaltgeschwindigkeit [m/s]
num_turbines = 6 # Anzahl Windkraftanlagen

# Batteriespeicher
battery_capacity = 12  # [MWh]
soc = 0.5  # State of Charge (SOC) initial [0-1]
initial_battery = soc * battery_capacity  # [MWh]
charging_rate = 10  # Lade-/Entladerate [MW]
discharging_rate = 10

# Verbraucher
yearly_demand = 2.7  #  Verbrauch eines Haushalts/Jahr [MWh] (Annahme)
num_houses = 39000  # Anzahl Haushalte

# Strompreis
export_factor = 0.25   # Anteil vom Spotpreis für Einspeisung
price_low_quant  = 0.30   # Regelbasiert: Niedrigpreis-Schwelle

# Parameter für Sliding Window
test_month = 5  # Startmonat für das Sliding Window (1 bis 5) (5)
test_timeframe = 3  # Datensatz kürzen [Tage] (7)
forecast_horizon = 1    # Länge des Fensters [Tage] (2)
stepsize = 1    # Schrittweite [Stunden] (1)

# RL Parameter
split = 80/20 # Train/Test Split
total_timesteps = 500  # Anzahl der Schritte für das Training des RL-Agenten

**Vorbereiten der Datensätze**

In [ ]:
df_weather_forecast = get_weather_forecast()
df_weather_real = get_weather_real()
df_spot_forecast, df_spot_real = get_spotprice_forecast()
df_demand_real = get_demand_forecast(num_houses, yearly_demand)

**Ausführen der Berechnungsfunktion**

In [ ]:
# Energie- und Bedarfsberechnung für Trainings- und Testdaten
# Berechnet PV-/Wind-Ertrag und merged alle relevanten Spalten.

df_energy_forecast = calculate_energy(
    df_weather_forecast, 
    solar_peak_power, num_modules, area_per_module, module_efficiency, tilt_angle, latitude, longitude, azimuth, albedo, 
    wind_peak_power, r, h, turbine_efficiency, cut_in, cut_out, num_turbines
)

df_forecast = df_energy_forecast.merge(
    df_spot_forecast[['datetime', 'spotprice']],
    on='datetime', how='left'
 ).merge(
    df_demand_real[['datetime', 'energy_demand']],
    on='datetime', how='left'
 ).reset_index(drop=True)

df_energy_real = calculate_energy(
    df_weather_real, 
    solar_peak_power, num_modules, area_per_module, module_efficiency, tilt_angle, latitude, longitude, azimuth, albedo, 
    wind_peak_power, r, h, turbine_efficiency, cut_in, cut_out, num_turbines
)

df_real = df_energy_real.merge(
    df_spot_real[['datetime', 'spotprice']],
    on='datetime', how='left'
 ).merge(
    df_demand_real[['datetime', 'energy_demand']],
    on='datetime', how='left'
 ).reset_index(drop=True)

# Optional: Daten anzeigen
# display(df_train.head()), display(df_test.head())

**Ausführen der Entscheidungs-Funktionen**

In [ ]:
# Regelbasierte Steuerung (Testdaten)
# Einfache Heuristik für Batterie- und Netzmanagement.

result_df_rule_forecast = rule_based_energy_flow(
    df_forecast, 
    battery_capacity, initial_battery, charging_rate, discharging_rate,
    export_factor, price_low_quant
    )

result_df_rule_real = rule_based_energy_flow(
    df_real, 
    battery_capacity, initial_battery, charging_rate, discharging_rate,
    export_factor, price_low_quant
    )

result_kpis_rule_forecast = calculate_kpis_valid(result_df_rule_forecast, df_spot_real, export_factor)
result_kpis_rule_real = calculate_kpis_valid(result_df_rule_real, df_spot_real, export_factor)

In [ ]:
# Mathematische Optimierung (Testdaten)
# Optimiert die Energieflüsse für das Testfenster mit Extremwertproblem.

result_df_opt_forecast = optimize_energy_flow(
    df_forecast, 
    battery_capacity, initial_battery, charging_rate, discharging_rate,
    export_factor, 
    forecast_horizon, stepsize
    )

result_df_opt_real = optimize_energy_flow(
    df_real, 
    battery_capacity, initial_battery, charging_rate, discharging_rate,
    export_factor, 
    forecast_horizon, stepsize
    )

result_kpis_opt_forecast = calculate_kpis_valid(result_df_opt_forecast, df_spot_real, export_factor)
result_kpis_opt_real = calculate_kpis_valid(result_df_opt_real, df_spot_real, export_factor)

In [ ]:
# Reinforcement Learning: Test mit gespeichertem Modell
# Lädt das trainierte Modell und testet es auf den Testdaten (Sliding Window, SOC-Übergabe).

from stable_baselines3 import DQN
from functions.deciding_functions.reinforcementlearning.reinforcementlearning_train import EMSenv

# Modell laden
model = DQN.load("model/rl_model_trained.zip")

# Testfunktion ausführen (Parameter wie beim Training)
env_kwargs = {
    'battery_capacity': battery_capacity,
    'charging_rate': charging_rate,
    'discharge_rate': discharging_rate,
    'export_price_factor': export_factor,
}

result_df_rl_forecast = reinforcement_learning_test(
    model,
    df_forecast,
    EMSenv,
    env_kwargs,
    forecast_horizon,
    stepsize,
    initial_battery
)

result_df_rl_real = reinforcement_learning_test(
    model,
    df_real,
    EMSenv,
    env_kwargs,
    forecast_horizon,
    stepsize,
    initial_battery
)

result_kpis_rl_forecast = calculate_kpis_valid(result_df_rl_forecast, df_spot_real, export_factor)
result_kpis_rl_real = calculate_kpis_valid(result_df_rl_real, df_spot_real, export_factor)

In [ ]:
get_results(result_df_rule_forecast, result_df_opt_forecast, result_df_rl_forecast, result_kpis_rule_forecast, result_kpis_opt_forecast, result_kpis_rl_forecast, export_factor)

In [ ]:
get_results(result_df_rule_real, result_df_opt_real, result_df_rl_real, result_kpis_rule_real, result_kpis_opt_real, result_kpis_rl_real, export_factor)